# Growth media

The availability of nutrients has a major impact on metabolic fluxes and `cobrapy` provides some helpers to manage the exchanges between the external environment and your metabolic model. In experimental settings the "environment" is usually constituted by the growth medium, ergo the concentrations of all metabolites and co-factors available to the modeled organism. However, constraint-based metabolic models only consider fluxes. Thus, you can not simply use concentrations since fluxes have the unit `mmol / [gDW h]` (concentration per gram dry weight of cells and hour). 

Also, you are setting an upper bound for the particular import flux and not the flux itself. There are some crude approximations. For instance, if you supply 1 mol of glucose every 24h to 1 gram of bacteria you might set the upper exchange flux for glucose to `1 mol / [1 gDW * 24 h]` since that is the nominal maximum that can be imported. There is no guarantee however that glucose will be consumed with that flux. Thus, the preferred data for exchange fluxes are direct flux measurements as the ones obtained from timecourse exa-metabolome measurements for instance. 

So how does that look in COBRApy? The current growth medium of a model is managed by the `medium` attribute.  

In [1]:
from cobra.io import load_model

model = load_model("textbook")
model.medium

Scaling...
 A: min|aij| =  1.000e+00  max|aij| =  1.000e+00  ratio =  1.000e+00
Problem data seem to be well scaled


{'EX_co2_e': 1000.0,
 'EX_glc__D_e': 10.0,
 'EX_h_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_pi_e': 1000.0}

This will return a dictionary that contains the upper flux bounds for all active exchange fluxes (the ones having non-zero flux bounds). Right now we see that we have enabled aerobic growth. You can modify a growth medium of a model by assigning a dictionary to `model.medium` that maps exchange reactions to their respective upper import bounds. For now let us enforce anaerobic growth by shutting off the oxygen import.

In [2]:
medium = model.medium
medium["EX_o2_e"] = 0.0
model.medium = medium

model.medium

{'EX_co2_e': 1000.0,
 'EX_glc__D_e': 10.0,
 'EX_h_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_pi_e': 1000.0}

As we can see oxygen import is now removed from the list of active exchanges and we can verify that this also leads to a lower growth rate.

In [3]:
model.slim_optimize()

0.21166294973530736

There is a small trap here. `model.medium` can not be assigned to directly. So the following will not work:

In [4]:
model.medium["EX_co2_e"] = 0.0
model.medium

{'EX_co2_e': 1000.0,
 'EX_glc__D_e': 10.0,
 'EX_h_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_pi_e': 1000.0}

As you can see `EX_co2_e` is not set to zero. This is because model.medium is just a copy of the current exchange fluxes. Assigning to it directly with `model.medium[...] = ...` will **not** change the model. You have to assign an entire dictionary with the changed import flux upper bounds:

In [5]:
medium = model.medium
medium["EX_co2_e"] = 0.0
model.medium = medium

model.medium  # now it worked

{'EX_glc__D_e': 10.0,
 'EX_h_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_pi_e': 1000.0}

Setting the growth medium also connects to the context manager, so you can set a specific growth medium in a reversible manner.

In [6]:
model = load_model("textbook")

with model:
    medium = model.medium
    medium["EX_o2_e"] = 0.0
    model.medium = medium
    print(model.slim_optimize())
print(model.slim_optimize())
model.medium

0.21166294973530736
0.8739215069684102


{'EX_co2_e': 1000.0,
 'EX_glc__D_e': 10.0,
 'EX_h_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_pi_e': 1000.0}

So the medium change is only applied within the `with` block and reverted automatically.

## Minimal media

In some cases you might be interested in the smallest growth medium that can maintain a specific growth rate, the so called "minimal medium". For this we provide the function `minimal_medium` which by default obtains the medium with the lowest total import flux. This function needs two arguments: the model and the minimum growth rate (or other objective) the model has to achieve.

In [7]:
from cobra.medium import minimal_medium

max_growth = model.slim_optimize()
minimal_medium(model, max_growth)

EX_glc__D_e    10.000000
EX_nh4_e        4.765319
EX_o2_e        21.799493
EX_pi_e         3.214895
dtype: float64

So we see that growth is actually limited by glucose import.

Alternatively you might be interested in a minimal medium with the smallest  number of active imports. This can be achieved by using the `minimize_components` argument (note that this uses a MIP formulation and will therefore be much slower).

In [8]:
minimal_medium(model, 0.1, minimize_components=True)

EX_glc__D_e    10.00000
EX_nh4_e        0.54528
EX_pi_e         0.36787
dtype: float64

When minimizing the number of import fluxes there may be many alternative solutions. To obtain several of those you can also pass a positive integer to `minimize_components` which will give you at most that many alternative solutions. Let us try that with our model and also use the `open_exchanges` argument which will assign a large upper bound to all import reactions in the model. The return type will be a `pandas.DataFrame`.

In [9]:
minimal_medium(model, 0.8, minimize_components=8, open_exchanges=True)

,0,1,2,3,4,5
EX_fru_e,0.000000,308.82944,0.00000,31.58596,0.000000,0.000000
EX_glc__D_e,0.000000,0.00000,308.82944,0.00000,31.157104,0.000000
EX_gln__L_e,0.000000,0.00000,0.00000,2.18112,0.000000,18.848678
EX_glu__L_e,277.588056,6.10840,6.10840,0.00000,0.000000,0.000000
EX_mal__L_e,0.000000,0.00000,0.00000,0.00000,0.000000,1000.000000
EX_nh4_e,0.000000,0.00000,0.00000,0.00000,4.362240,0.000000
EX_o2_e,500.000000,0.00000,0.00000,0.00000,0.000000,0.000000
EX_pi_e,46.944976,2.94296,2.94296,2.94296,2.942960,12.583458


So there are 4 alternative solutions in total. One aerobic and three anaerobic ones using different carbon sources.

## Boundary reactions

Apart from exchange reactions there are other types of boundary reactions such as demand or sink reactions. `cobrapy` uses various heuristics to identify those and they can be accessed by using the appropriate attribute.

For exchange reactions:

In [10]:
ecoli = load_model("iJO1366")
ecoli.exchanges[0:5]

[<Reaction EX_12ppd__R_e at 0x7f88afe3dcf8>,
 <Reaction EX_12ppd__S_e at 0x7f88afe7c240>,
 <Reaction EX_14glucan_e at 0x7f88afe46240>,
 <Reaction EX_15dap_e at 0x7f88afe46278>,
 <Reaction EX_23camp_e at 0x7f88afe463c8>]

For demand reactions:

In [11]:
ecoli.demands

[<Reaction DM_4crsol_c at 0x7f88afe3d208>,
 <Reaction DM_5drib_c at 0x7f88afe3d2e8>,
 <Reaction DM_aacald_c at 0x7f88afe3d400>,
 <Reaction DM_amob_c at 0x7f88afe3d630>,
 <Reaction DM_mththf_c at 0x7f88afe3d7b8>,
 <Reaction DM_oxam_c at 0x7f88afe3d8d0>]

For sink reactions:

In [12]:
ecoli.sinks

[]

All boundary reactions (any reaction that consumes or introduces mass into the system) can be obtained with the `boundary` attribute:

In [13]:
ecoli.boundary[0:10]

[<Reaction DM_4crsol_c at 0x7f88afe3d208>,
 <Reaction DM_5drib_c at 0x7f88afe3d2e8>,
 <Reaction DM_aacald_c at 0x7f88afe3d400>,
 <Reaction DM_amob_c at 0x7f88afe3d630>,
 <Reaction DM_mththf_c at 0x7f88afe3d7b8>,
 <Reaction DM_oxam_c at 0x7f88afe3d8d0>,
 <Reaction EX_12ppd__R_e at 0x7f88afe3dcf8>,
 <Reaction EX_12ppd__S_e at 0x7f88afe7c240>,
 <Reaction EX_14glucan_e at 0x7f88afe46240>,
 <Reaction EX_15dap_e at 0x7f88afe46278>]